# 4주차 Double DQN

## [핵심 목표]
* Double DQN의 개념을 이해하고, Q값 과대 추정(Overestimation) 문제를 완화하는 방법을 학습
* Double DQN을 기존 DQN 코드에 적용하는 방법을 이해

## [Q값 과대 추정 (Overestimation) 문제]

* DQN은 최대 Q값을 직접 사용하기 때문에, 노이즈나 학습 초기 불안정성으로 인해 실제보다 높은 Q값을 선택하는 경향이 있음
* 이로 인해 정책이 왜곡될 수 있음

### 1. Q값 과대 추정 문제 예시: 게임 상황

* **문제**: 간단한 미로 탈출 게임
* **목표**: 미로를 탈출하여 보상을 얻는 것
* **에이전트**: DQN 알고리즘을 사용하여 학습하는 캐릭터


### 2. DQN의 과대 추정 문제 발생 과정

* 학습 초기 단계에서 에이전트는 우연히 미로의 특정 구역에 도달했을 때 높은 Q값을 얻음
* 이 높은 Q값은 실제 보상보다 과대 추정된 값일 수 있음
  * 이 구역에 도달한 것이 가치가 있어서 Q값이 높은 것이 아닐 수 있음
  * 달리 말하면, 아직 다른 가치 있는 구역을 모르는 것일 뿐일 수 있음
* 에이전트는 이 과대 추정된 Q값을 '진짜 가치가 있다'라고 믿고 해당 구역을 최적의 경로라고 판단

### 3. 정책 왜곡 및 문제점

* 실제로 해당 구역은 최적의 경로가 아닐 수 있고, 더 짧거나 안전한 다른 경로가 존재할 수 있음
* 하지만 에이전트는 과대 추정된 Q값 때문에 계속해서 해당 구역으로 향하는 잘못된 결정을 내림
* 결과적으로 에이전트는 최적의 경로를 학습하지 못하고 미로 탈출에 실패하거나 더 오랜 시간이 걸리게 됨

## [Double DQN 아이디어]

* 행동 선택과 Q값 평가를 분리
* 기존 DQN: 메인 네트워크를 사용하여 행동을 선택하고, 타겟 네트워크를 사용하여 선택된 행동의 Q값을 평가
* Double DQN:
  * 메인 네트워크를 사용하여 최적 행동 $a^*$ 을 선택
  * 타겟 네트워크를 사용하여 선택된 행동 $a^*$ 의 Q값을 평가

## [Double DQN 학습 업데이트]

* Temporal Difference Tagert (TD Target):
  * 기존 DQN: $r + \gamma \underset{a'}{max\;} Q(s', a'; \theta^-)$
  * Double DQN: $r + \gamma Q(s', \underset{a}{arg \text{ } max \text{ }} Q(s', a; \theta); \theta^-)$
    * $r$: 즉각적인 보상
    * $\gamma$: 할인율
    * $s'$: 다음 상태
    * $\theta$: 메인 네트워크의 파라미터
    * $\theta^-$: 타겟 네트워크의 파라미터
    * $\underset{a}{arg \text{ } max \text{ }} Q(s', a; \theta)$: 메인 네트워크를 사용하여 Q값이 최대가 되는 행동 $a$ 선택
    * $Q(s', \underset{a}{arg \text{ } max \text{ }} Q(s', a; \theta); \theta^-)$: 선택된 행동 $a$에 대한 타겟 네트워크의 Q값 평가
* 손실 함수 (Loss Function): TD 에러를 최소화하는 방향으로 학습
  * TD 에러: 현재 네트워크 예측과 실제 타겟 간의 차이
  * $Error_{TD, DQN} = Q(s, a; \theta) - (r + \gamma \underset{a'}{max} \text{ } Q(s', a'; \theta^-))$
    * $\underset{a'}{max} \text{ } Q(s', a'; \theta^-)$: 최대 Q값의 선택과 Q값 평가에 동일한 네트워크($\theta^-$) 사용
    * Q값 과대 추정 문제 발생
  * $Error_{TD, DDQN} = Q(s, a; \theta) - (r + \gamma Q(s', \underset{a}{arg \text{ } max \text{ }} Q(s', a; \theta); \theta^-))$
    * $\underset{a}{arg \text{ } max \text{ }} Q(S', a'; \theta)$: 메인 네트워크($\theta$)를 사용하여 다음 상태($S'$)에서 가장 높은 Q값을 가지는 행동($a'$)을 선택
    * $Q(S', \underset{a}{arg \text{ } max \text{ }} Q(S', a'; \theta); \theta')$: 선택된 행동($a'$)의 Q값을 타겟 네트워크($θ'$)를 사용해 평가
    * 이 차이가 Q값 과대 추정 문제를 완화하는 데 중요한 역할을 함
* 업데이트: 손실 함수를 최소화하는 방향으로 메인 네트워크의 파라미터를 업데이트





## [실습: Double DQN을 이용한 CartPole 학습]

### 1. 환경 설정

* `gym`: CartPole 환경 제공
* `torch`: 딥러닝 모델 및 연산 처리
* `numpy`: 배열 및 수치 연산
* `collections.deque`: 경험 리플레이 버퍼 구현

In [ ]:
import gym
import random
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from collections import deque

env = gym.make('CartPole-v1')
state_size = env.observation_space.shape[0]
action_size = env.action_space.n

/usr/local/lib/python3.11/dist-packages/gym/core.py:317: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/usr/local/lib/python3.11/dist-packages/gym/wrappers/step_api_compatibility.py:39: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(


### 2. DQN 모델 정의

* `DQNAgent` 클래스: 신경망 모델 정의
* 3개의 완전 연결 계층 (`nn.Linear`) 사용
* ReLU 활성화 함수 적용 (`torch.relu`)
* 입력: 상태, 출력: 각 행동의 Q값

In [ ]:
class DQNAgent(nn.Module):
    def __init__(self, state_size, action_size):
        super(DQNAgent, self).__init__()
        self.fc1 = nn.Linear(state_size, 24)
        self.fc2 = nn.Linear(24, 24)
        self.fc3 = nn.Linear(24, action_size)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        return self.fc3(x)

### 3. 하이퍼파라미터 설정

* 학습률 (`learning_rate`), 할인율 (`gamma`), 탐험 확률 (`epsilon`) 등 설정
* 미니배치 크기 (`batch_size`), 메모리 크기 (`memory_size`) 설정
* 타겟 네트워크 업데이트 주기 (`target_update_frequency`) 설정

In [ ]:
learning_rate = 0.001
gamma = 0.99  # 할인율
epsilon = 1.0  # 초기 탐험 확률
epsilon_min = 0.01
epsilon_decay = 0.999
batch_size = 64
memory_size = 10000
target_update_frequency = 10  # 타겟 네트워크 업데이트 주기

### 4. DQN 에이전트 및 타겟 네트워크 생성

* `DQNAgent` 모델 인스턴스 생성 (agent, target_agent)
* 타겟 네트워크 초기화 (agent 네트워크의 가중치 복사)
* Adam optimizer 및 MSE loss function 설정

In [ ]:
agent = DQNAgent(state_size, action_size)
target_agent = DQNAgent(state_size, action_size)
target_agent.load_state_dict(agent.state_dict())
optimizer = optim.Adam(agent.parameters(), lr=learning_rate)
criterion = nn.MSELoss()

### 5. 경험 리플레이 버퍼 초기화

* `deque`를 사용하여 경험 리플레이 버퍼 (`memory`) 초기화

In [ ]:
memory = deque(maxlen=memory_size)

### 6. 학습 함수 (`train`)

* 경험 리플레이 버퍼에서 미니배치 샘플링
* 상태, 행동, 보상, 다음 상태, 종료 여부 추출
* **Double DQN 핵심**: agent 네트워크에서 다음 상태의 최대 Q-값을 가지는 행동 선택, target 네트워크에서 선택된 행동의 Q-값 계산
* 예측 Q-값과 타겟 Q-값 비교하여 손실 함수 계산
* 역전파 및 가중치 업데이트

In [ ]:
def train(batch_size):
    minibatch = random.sample(memory, batch_size)
    states = torch.tensor(np.array([transition[0] for transition in minibatch]), dtype=torch.float32)
    actions = torch.tensor(np.array([transition[1] for transition in minibatch]), dtype=torch.long)
    rewards = torch.tensor(np.array([transition[2] for transition in minibatch]), dtype=torch.float32)
    next_states = torch.tensor(np.array([transition[3] for transition in minibatch]), dtype=torch.float32)
    dones = torch.tensor(np.array([transition[4] for transition in minibatch]), dtype=torch.float32)

    # Double DQN 핵심: agent 네트워크에서 다음 상태의 최대 Q-값을 가지는 행동 선택
    next_actions = agent(next_states).argmax(1).unsqueeze(1)
    # target 네트워크에서 선택된 행동의 Q-값 계산
    target_q_values = target_agent(next_states).gather(1, next_actions).squeeze()
    targets = rewards + gamma * target_q_values * (1 - dones)

    predicted_q_values = agent(states).gather(1, actions.unsqueeze(1)).squeeze()
    loss = criterion(predicted_q_values, targets)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

### 7. 학습 루프

* `episodes`만큼 에피소드 반복
* 각 에피소드마다 환경 초기화 및 상태 설정
* ε-탐욕적 정책으로 행동 선택
* 환경에서 행동 실행 후 다음 상태, 보상, 종료 여부 얻음
* 경험 리플레이 버퍼에 경험 추가
* 버퍼 크기가 `batch_size` 이상이면 학습 함수 호출
* 타겟 네트워크 업데이트 및 탐험 확률 감소
* 에피소드 결과 출력

In [ ]:
# 학습 루프
episodes = 100
for e in range(episodes):
    state = env.reset()
    state = torch.tensor(state, dtype=torch.float32)
    done = False
    total_reward = 0

    while not done:
        if random.random() < epsilon:
            action = env.action_space.sample()
        else:
            action = agent(state).argmax().item()
        next_state, reward, done, _ = env.step(action)
        next_state = torch.tensor(next_state, dtype=torch.float32)

        total_reward += reward
        memory.append((state, action, reward, next_state, done))

        if len(memory) >= batch_size:
            train(batch_size)

        state = next_state
        if e % target_update_frequency == 0:
            target_agent.load_state_dict(agent.state_dict())

        if epsilon > epsilon_min:
            epsilon *= epsilon_decay

    print(f"Episode {e + 1}/{episodes}, Total Reward: {total_reward}, Epsilon: {epsilon:.3f}")

Episode 1/100, Total Reward: 21.0, Epsilon: 0.979
Episode 2/100, Total Reward: 30.0, Epsilon: 0.950
Episode 3/100, Total Reward: 27.0, Epsilon: 0.925
Episode 4/100, Total Reward: 16.0, Epsilon: 0.910
Episode 5/100, Total Reward: 29.0, Epsilon: 0.884
Episode 6/100, Total Reward: 8.0, Epsilon: 0.877
Episode 7/100, Total Reward: 20.0, Epsilon: 0.860
Episode 8/100, Total Reward: 13.0, Epsilon: 0.849
Episode 9/100, Total Reward: 18.0, Epsilon: 0.834
Episode 10/100, Total Reward: 20.0, Epsilon: 0.817
Episode 11/100, Total Reward: 18.0, Epsilon: 0.802
Episode 12/100, Total Reward: 40.0, Epsilon: 0.771
Episode 13/100, Total Reward: 19.0, Epsilon: 0.756
Episode 14/100, Total Reward: 42.0, Epsilon: 0.725
Episode 15/100, Total Reward: 49.0, Epsilon: 0.691
Episode 16/100, Total Reward: 49.0, Epsilon: 0.658
Episode 17/100, Total Reward: 92.0, Epsilon: 0.600
Episode 18/100, Total Reward: 22.0, Epsilon: 0.587
Episode 19/100, Total Reward: 40.0, Epsilon: 0.564
Episode 20/100, Total Reward: 38.0, Epsil

### 8. 학습된 에이전트 검증 (렌더링 with GIF)

In [ ]:
import imageio
import os

frames = []
state = env.reset()
state = torch.tensor(state, dtype=torch.float32)
done = False
for i in range(3000): # 3000 steps simulation
  action = agent(state).argmax().item()
  next_state, reward, done, _ = env.step(action)
  next_state = torch.tensor(next_state, dtype=torch.float32)
  state = next_state

  if i % 10 == 0: # Render every 10 steps
    frames.append(env.render(mode='rgb_array'))

  if done:
    break

imageio.mimsave('cartpole_simulation_doubleDQN.gif', frames, duration=33) # save to gif file
print("GIF saved as cartpole_simulation.gif")


GIF saved as cartpole_simulation.gif
